## Import Dependencies

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import math
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import vgg16

In [ ]:
# Device Agnostic Code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Initialization

In [ ]:
# View first few rows of train csv
train_df = pd.read_csv("/kaggle/input/ripik-hackfest/train/train/train.csv")
train_df.head()

In [ ]:
# Create dictionary for sparse label to corresponding item for plotting
labels_dict = {
    0: "crack",
    1: "scratch",
    2: "tire flat",
    3: "dent",
    4: "glass shatter",
    5: "lamp broken"
}

In [ ]:
# View data type of train_df
train_df.info()

In [ ]:
# Adjust the labels to start at 0
train_df["label"] = train_df["label"] - 1

In [ ]:
# View class label distribution
train_df["label"].value_counts()

The big imbalance in class would indicate a Random Weighted Sampler, data augmentation, or class weights may be needed

## Sample Data Visualization

In [ ]:
# Function to View a Few Images
def view_sample_images(input_image_dir, dataframe, n_samples=5):
    sample_df = dataframe.sample(n_samples)
    num_rows = math.ceil(n_samples / 5)
    num_cols = 5
    fig = plt.figure(figsize = (15,10))
    for i in range(n_samples):
        ax = plt.subplot(num_rows, num_cols, i+1)
        file_name = os.path.join(input_image_dir, sample_df.iloc[i]["filename"])
        class_label = sample_df.iloc[i]["label"]
        class_category = labels_dict[class_label]
        img = cv2.imread(file_name)
        ax.imshow(img)
        ax.set_title(f"Class: {class_label} ({class_category})")
    plt.tight_layout()
    plt.show()

In [ ]:
# View five images
N_SAMPLES = 5
view_sample_images("/kaggle/input/ripik-hackfest/train/train/images", train_df, N_SAMPLES)

## Data Preparation

In [ ]:
# Create Custom Dataset
class VehicleImageLoader(Dataset):
    def __init__(self, dataframe, root_dir, transforms = None):
        self.annotations = dataframe
        self.transforms = transforms
        self.root_dir = root_dir
    
    def __len__(self):
        return len(self.annotations)
    
    def  __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx]["filename"])
        img = cv2.imread(img_path)
        label = self.annotations.iloc[idx]["label"]
        
        if self.transforms:
            img = self.transforms(img)
            
        return (img, label)

In [ ]:
# Create transforms pipeline
transform_pipeline = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [ ]:
# Create dataset
dataset = VehicleImageLoader(train_df, "/kaggle/input/ripik-hackfest/train/train/images", transforms = transform_pipeline)

In [ ]:
# Partition the train dataset into train and validation
TRAIN_SPLIT = 0.80

num_train = int(TRAIN_SPLIT * len(dataset))
num_validation = len(dataset) - num_train
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [num_train, num_validation])

In [ ]:
# Create the dataloaders
BATCH_SIZE = 32

train_dataloader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(dataset = val_dataset, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
# View first batch
data_iter = iter(train_dataloader)
batch = next(data_iter)
batch[0].shape, batch[1].shape

## Data Modeling

In [ ]:
# Initialize Pretrained Model
model = vgg16(weights="VGG16_Weights.DEFAULT")

In [ ]:
model

### Fine-tuning model

In [ ]:
# Change classifier final output
num_classes = train_df["label"].nunique()
model.classifier[6] = nn.Linear(in_features = 4096, out_features = num_classes, bias = True)

In [ ]:
# Sent model to device
model.to(device)

### Optimizer and Loss Function

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "min", factor = 0.2, patience = 2, verbose = True)

In [ ]:
def check_accuracy(y_pred_logits, y_true):
    predictions = torch.argmax(y_pred_logits, dim = 1)
    accuracy = accuracy_score(y_true.cpu().numpy(), predictions.cpu().numpy())
    
    return accuracy

In [ ]:
# Basic training function
# for _ in range(4):
#     X, y = batch
#     y_pred = model(X)
#     loss = criterion(y_pred, y)
#     print(loss)
#     accuracy = check_accuracy(y_pred, y)
#     print("Accuracy:",accuracy)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

### Creating Training Function

In [ ]:
def train_step(model, train_loader, val_loader, optimizer, criterion, num_epochs, total_patience = 5, device = device):
    train_losses = []
    val_losses = []
    train_accuracy = []
    val_accuracy = []
    
    best_val_loss = float("inf")
    patience = 0
    
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        total_train_acc = 0
        loop = tqdm(enumerate(train_loader), total = len(train_loader), leave = False)
        for batch, (X, y) in loop:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # Get predictions
            y_pred = model(X)
            
            # Calulcate loss
            train_loss = criterion(y_pred, y)
            total_train_loss += train_loss.item()
            
            # Get accuracy
            batch_accuracy = check_accuracy(y_pred, y)
            total_train_acc += batch_accuracy
            
            # Zero optimizer
            optimizer.zero_grad()
            
            # Backpropagation
            train_loss.backward()
            
            # Gradient Descent
            optimizer.step()
            
            # Update Progress Bar
            loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
            loop.set_postfix(loss = train_loss.item(), acc = batch_accuracy)
                 
        # Calculate total loss value (average of all batches)   
        average_train_loss = total_train_loss / len(train_loader)
        train_losses.append(average_train_loss)
        
        # calculate total accuracy
        average_train_accuracy = total_train_acc / len(train_loader)
        train_accuracy.append(average_train_accuracy)
        
        # Check Validation
        model.eval()
        total_val_loss = 0
        total_val_acc = 0
        with torch.inference_mode():
            val_loop = tqdm(enumerate(val_loader), total = len(val_dataloader), leave = False)
            for batch, (X,y) in val_loop:
                X, y = X.to(device), y.to(device)
                
                y_pred = model(X)
                
                val_loss = criterion(y_pred, y)
                total_val_loss += val_loss.item()
                
                batch_accuracy = check_accuracy(y_pred, y)
                total_val_acc += batch_accuracy
                
                val_loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
                val_loop.set_postfix(loss = val_loss.item(), acc = batch_accuracy)
                  
        average_val_loss = total_val_loss / len(val_loader)
        val_losses.append(average_val_loss)
        
        average_val_accuracy = total_val_acc / len(val_loader)
        val_accuracy.append(average_val_accuracy)
        
        
        # Early Stopping and Model Checkpoint
        if average_val_loss < best_val_loss:
            print(f"Validation Loss Improved From: {best_val_loss} to {average_val_loss}")
            best_val_loss = average_val_loss
            
            # Save entire model
            torch.save(model.state_dict(), "vehicle_classification_weights.pth")
            print("Saved Weights to vehicle_classification_weights.pth")
            
            patience = 0
        else:
            print("Validation Loss Did not Improve")
            patience += 1
            
            if patience == total_patience:
                print("End Training: Early Stopping")
                print(f"Epoch: {epoch+1} | Train Loss: {average_train_loss:.4f} | Validation Loss: {average_val_loss:.4f} | Train Accuracy: {average_train_accuracy:.4f} | Validation Accuracy: {average_val_accuracy:.4f}")
                break
        scheduler.step(average_val_loss)       
        print(f"Epoch: {epoch+1} | Train Loss: {average_train_loss:.4f} | Validation Loss: {average_val_loss:.4f} | Train Accuracy: {average_train_accuracy:.4f} | Validation Accuracy: {average_val_accuracy:.4f}")
    return {"Train Accuracy" : train_accuracy, "Validation Accuracy" : val_accuracy, "Train Loss" : train_losses, "Validation Loss" : val_losses}

In [ ]:
# Deterministic Behavior
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Training
NUM_EPOCHS = 20
history = train_step(model, train_dataloader, val_dataloader, optimizer, criterion, NUM_EPOCHS)

### Save Model and Other Parameters

In [ ]:
# Save loss and accuracy
torch.save(history, "vehicle_classification_history.pth")

In [ ]:
history

## Model Performance

In [ ]:
# Load the model
model = vgg16(weights="VGG16_Weights.DEFAULT")
model.classifier[6] = nn.Linear(in_features = 4096, out_features = num_classes, bias = True)
model.load_state_dict(torch.load("vehicle_classification_weights.pth"))

# Load the history
history = torch.load("vehicle_classification_history.pth")

### Model Loss and Accuracy

In [ ]:
# Plot the loss and accuracies
fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (10,6))

# Loss Values
axes[0].plot(history["Train Loss"], label = "Train Loss")
axes[0].plot(history["Validation Loss"], label = "Validation Loss")
axes[0].set_title("Model Loss")
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Loss")
axes[0].legend()

axes[1].plot(history["Train Accuracy"], label = "Train Accuracy")
axes[1].plot(history["Validation Accuracy"], label = "Validation Accuracy")
axes[1].set_title("Model Accuracy")
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Accuracy")
axes[1].legend()

plt.tight_layout()
plt.show()

### Confusion Matrix

In [ ]:
# Get predictions
model.to(device)
model.eval()
true_labels = []
pred_labels = []

with torch.inference_mode():
    for batch, (X, y) in enumerate(val_dataloader):
        X, y = X.to(device), y.to(device)
        logits = model(X)
        prediction = torch.argmax(logits, dim = 1)
        
        # Append to lists
        true_labels.extend(y.cpu().numpy())
        pred_labels.extend(prediction.cpu().numpy())

In [ ]:
# Convert lists to NumPy array
true_labels = np.array(true_labels)
pred_labels = np.array(pred_labels)

In [ ]:
accuracy_score(true_labels, pred_labels)

In [ ]:
# Calculate the confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)

In [ ]:
# Plot the confusion matrix as a seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

## Make Predictions

In [ ]:
# Read in test csv
test_df = pd.read_csv("/kaggle/input/ripik-hackfest/test/test/test.csv")
test_df.head()

In [ ]:
# Create full paths for all images
test_images_dir = "/kaggle/input/ripik-hackfest/test/test/images/"
test_df["filename"] = test_images_dir + test_df["filename"]

### Make Predictions

In [ ]:
def make_predictions(dataframe, model, transforms, num_preds = 5):
    model.to("cpu")
    sample_df = dataframe.sample(num_preds)
    num_rows = math.ceil(num_preds / 5)
    num_cols = 5
    fig = plt.figure(figsize = (15,10))
    for i in range(num_preds):
        image_path = sample_df.iloc[i]["filename"]
        img = cv2.imread(image_path)
        ax = plt.subplot(num_rows, num_cols, i+1)
        img = transforms(img)
        model.eval()
        with torch.inference_mode():
            pred = model(img.unsqueeze(0))
            pred = torch.argmax(pred, dim = 1)
        ax.imshow(np.transpose(img, [1,2,0]))
        ax.set_title(f"Predicted: {labels_dict[pred.item()]}")
        

In [ ]:
make_predictions(test_df, model, transform_pipeline)